In [1]:
import numpy as np
import pandas as pd
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

C:\Users\HSS\Documents\Anaconda3\envs\tensorflow\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\Users\HSS\Documents\Anaconda3\envs\tensorflow\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
article_df = pd.read_csv('article.csv')
article_df.tail(2)

,Link,Text,Title
598,https://news.naver.com/main/read.nhn?mode=LSD&...,'정보 위변조 불가' 및 '추적 가능' 등 특성 활용(지디넷코리아=유효정 기자)중...,"中 공공·금융업계 ""블록체인으로 코로나19 퇴치"""
599,https://news.naver.com/main/read.nhn?mode=LSD&...,남궁 대표 설립 주도…'아키에이지 워크' 흥행 여부 관심[아이뉴스24 문영수 기자]...,제2의 포켓몬고 나오나 …라이프엠엠오 눈길 끄는 이유


In [15]:
article_df['Category'] = [100]*100 + [101]*100 + [102]*100 + [103]*100 + [104]*100 + [105]*100

In [28]:
#데이터전처리 : 데이터셋 분리
X_train, X_test, Y_train, Y_test = train_test_split(article_df.Text, article_df.Category, test_size = 0.1, random_state = 1)

In [31]:
#모델만들기
clf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB(alpha = 0.1)),
    
    
])

In [32]:
#학습
model = clf.fit(X_train.values.astype('str'), Y_train.values)

In [36]:
y_pred = model.predict(X_test)

In [48]:
print(classification_report(Y_test, y_pred))

             precision    recall  f1-score   support

        100       0.50      0.70      0.58        10
        101       0.60      0.75      0.67         8
        102       0.60      0.60      0.60        10
        103       0.60      0.43      0.50         7
        104       1.00      0.69      0.81        16
        105       0.80      0.89      0.84         9

avg / total       0.72      0.68      0.69        60



#### 모델 사용하기

In [49]:
categories = {
    100 : '정치',
    101 : '경제',
    102 : '사회',
    103 : '생활/문화',
    104 : '세계',
    105 : 'IT/과학'
}

In [60]:
contents=  [
    '트럼프 대통령은 UN 제재를 선언하였다. 문재인 대통령은 이에 성명을 발표했다.',
    '요즘 환율 주가 예측이 불가능하다. '
]

In [61]:
datas = {
    'content': contents,
    'category_code' : model.predict(contents)
}

In [62]:
result = pd.DataFrame(datas)

In [100]:
result['category'] = result.category_code.apply(lambda x: categories[x])
result['proba'] = result.content.apply(lambda x :round(max(model.predict_proba([x])[0]),2))

In [102]:
result

,category_code,content,category,proba
0,100,트럼프 대통령은 UN 제재를 선언하였다. 문재인 대통령은 이에 성명을 발표했다.,정치,0.35
1,101,요즘 환율 주가 예측이 불가능하다.,경제,0.40


In [103]:
#모델 저장
pickle.dump(model, open('clf.pkl', 'wb'))